In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.master("local[1]").appName("skew_salt_1").getOrCreate()

In [2]:
spark

In [3]:
import pandas as pd
import numpy as np

In [4]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

In [5]:
length = 100
names = np.random.choice(['Bob', 'James', 'Marek', 'Johannes', None], length)
amounts = np.random.randint(0, 1000000, length)

In [6]:
country = np.random.choice(
	['United Kingdom', 'Poland', 'USA', 'Germany', 'Russia'],
	length,
	p = [0.05, 0.05, 0.8, 0.05, 0.05]
)

In [7]:
#spark.conf.set("spark.sql.execution.arrow.enabled","true")
data = pd.DataFrame({'name': names, 'amount': amounts, 'country': country})
#df = spark.createDataFrame(data).repartition('country')

In [8]:
countries = spark.createDataFrame(pd.DataFrame({
	'id': [11, 12, 13, 14, 15],
	'country': ['United Kingdom', 'Poland', 'USA', 'Germany', 'Russia']
}))

C:\pyspark\spark-3.3.0-bin-hadoop3\python\pyspark\sql\pandas\conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
C:\pyspark\spark-3.3.0-bin-hadoop3\python\pyspark\sql\pandas\conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [10]:
df = spark.createDataFrame(data)
df.groupBy(spark_partition_id()).count().orderBy(desc("count")).show()

+--------------------+-----+
|SPARK_PARTITION_ID()|count|
+--------------------+-----+
|                   0|  100|
+--------------------+-----+



In [11]:
#df.groupBy(spark_partition_id()).count().orderBy(desc("count")).show()

In [12]:
#df.groupBy(spark_partition_id()).count().orderBy(desc("count")).show()

In [15]:
df.groupBy(col('country')).agg(count('*').alias('count')).orderBy('count').show()

+--------------+-----+
|       country|count|
+--------------+-----+
|       Germany|    3|
|United Kingdom|    3|
|        Russia|    4|
|           USA|   90|
+--------------+-----+



In [16]:
countries.show()

+---+--------------+
| id|       country|
+---+--------------+
| 11|United Kingdom|
| 12|        Poland|
| 13|           USA|
| 14|       Germany|
| 15|        Russia|
+---+--------------+



In [31]:
from pyspark.sql.types import *

In [34]:
df1 = df.withColumn("random_col",(round(rand()*200)).cast(IntegerType()))

In [42]:
df1.select('random_col').agg(max('random_col'),min('random_col')).show()

+---------------+---------------+
|max(random_col)|min(random_col)|
+---------------+---------------+
|            200|              2|
+---------------+---------------+



In [57]:
max_number = df1.select('random_col').agg(max('random_col').alias('maximum')).first()['maximum']+1
max_number

201

In [54]:
min_number = df1.select('random_col').agg(min('random_col').alias('minimum')).first()['minimum']
min_number

2

In [ ]:
#np.random.randint(1, 200 - 1)

In [62]:
countries_salt = countries.withColumn('salt',explode(array([lit(i) for i in range(min_number,max_number)])))

In [64]:
countries_salt.show()

+---+--------------+----+
| id|       country|salt|
+---+--------------+----+
| 11|United Kingdom|   2|
| 11|United Kingdom|   3|
| 11|United Kingdom|   4|
| 11|United Kingdom|   5|
| 11|United Kingdom|   6|
| 11|United Kingdom|   7|
| 11|United Kingdom|   8|
| 11|United Kingdom|   9|
| 11|United Kingdom|  10|
| 11|United Kingdom|  11|
| 11|United Kingdom|  12|
| 11|United Kingdom|  13|
| 11|United Kingdom|  14|
| 11|United Kingdom|  15|
| 11|United Kingdom|  16|
| 11|United Kingdom|  17|
| 11|United Kingdom|  18|
| 11|United Kingdom|  19|
| 11|United Kingdom|  20|
| 11|United Kingdom|  21|
+---+--------------+----+
only showing top 20 rows



In [67]:
#salt_join
salt_join = df1.join(countries_salt,(df1.country == countries_salt.country) & (df1.random_col == countries_salt.salt),'inner')

In [68]:
salt_join.show()

+--------+------+-------+----------+---+-------+----+
|    name|amount|country|random_col| id|country|salt|
+--------+------+-------+----------+---+-------+----+
|   James|177267|Germany|        80| 14|Germany|  80|
|   James|129971|Germany|       127| 14|Germany| 127|
|   James|999630|Germany|       134| 14|Germany| 134|
|    null|401923| Russia|        43| 15| Russia|  43|
|   James|399181| Russia|        60| 15| Russia|  60|
|     Bob|791559| Russia|        84| 15| Russia|  84|
|     Bob|508072| Russia|       124| 15| Russia| 124|
|     Bob|992981|    USA|         2| 13|    USA|   2|
|Johannes|818971|    USA|         6| 13|    USA|   6|
|     Bob|608366|    USA|         9| 13|    USA|   9|
|     Bob|545827|    USA|        13| 13|    USA|  13|
|    null|382460|    USA|        15| 13|    USA|  15|
|   Marek|247079|    USA|        15| 13|    USA|  15|
|Johannes|891902|    USA|        16| 13|    USA|  16|
|     Bob| 38788|    USA|        16| 13|    USA|  16|
|    null|527968|    USA|   

In [69]:
salt_join.count()

100

In [70]:
#df_join
df_join = df.join(countries,'country','inner')

In [71]:
df_join.show()

+-------+--------+------+---+
|country|    name|amount| id|
+-------+--------+------+---+
|Germany|   James|999630| 14|
|Germany|   James|177267| 14|
|Germany|   James|129971| 14|
| Russia|     Bob|791559| 15|
| Russia|   James|399181| 15|
| Russia|    null|401923| 15|
| Russia|     Bob|508072| 15|
|    USA|   James|378797| 13|
|    USA|     Bob|992981| 13|
|    USA|Johannes|653504| 13|
|    USA|Johannes|818971| 13|
|    USA|     Bob|608366| 13|
|    USA|   Marek|831918| 13|
|    USA|    null|631175| 13|
|    USA|   Marek|553477| 13|
|    USA|    null|170128| 13|
|    USA|     Bob|170951| 13|
|    USA|     Bob|973444| 13|
|    USA|   Marek|491460| 13|
|    USA|   James|791681| 13|
+-------+--------+------+---+
only showing top 20 rows



In [72]:
salt_join.groupBy(spark_partition_id()).count().show()

+--------------------+-----+
|SPARK_PARTITION_ID()|count|
+--------------------+-----+
|                   0|  100|
+--------------------+-----+



In [73]:
df_join.groupBy(spark_partition_id()).count().show()

+--------------------+-----+
|SPARK_PARTITION_ID()|count|
+--------------------+-----+
|                   0|  100|
+--------------------+-----+

